In [1]:
import os
import pandas as pd
import numpy as np
from quantities import ns
from neo.core import SpikeTrain
from elephant.statistics import isi, cv, mean_firing_rate
import matplotlib.pyplot as plt


def load_data(recording, data_dir, verbose):
    if verbose:
        print('Loading data:\t{}'.format(recording))
    path = ''.join([os.path.join(data_dir, recording, recording), '.csv'])
    return pd.read_csv(path)


def manipulate_df(df):
    df['spike'] = 1
    df['time'] = pd.to_timedelta(df['time'], unit='s')
    return df


def create_time_series(df):
    df = df.pivot_table(index='time',
                        columns='spike_cluster',
                        values='spike',
                        aggfunc='count')
    return df


def get_condition_times(df, experiment):
    if experiment == 'DREADD':
        max_time = df[df['condition'] == 'CNO']['time'].iloc[-1].total_seconds()
        n_conditions = 1
    if experiment == 'CIT':
        max_time_cit = df[df['condition'] == 'CIT']['time'].iloc[-1].total_seconds()
        if 'WAY' in df['condition'].values:
            max_time_way = df[df['condition'] == 'WAY']['time'].iloc[-1].total_seconds()
            n_conditions = 2
        else:
            max_time_way = max_time_cit
            n_conditions = 1
        max_time = list(zip(['CIT', 'WAY'], [max_time_cit, max_time_way]))
    return max_time, n_conditions


def calculate_neuron_cov(col, num_mins_per_bin, total_time):
    num_bins = np.int(total_time / num_mins_per_bin)
    col_bins = np.array_split(col, num_bins)
    cv_isis = pd.Series(np.zeros(num_bins))

    for ind, col_bin in enumerate(col_bins):
        spike_times = pd.to_numeric(col_bin[col_bin.notnull()].index.values)
        try:
            spike_train = SpikeTrain(times=spike_times,
                                     t_stop=spike_times[-1],
                                     units=ns)
            cv_isi = cv(isi(spike_train))
        except IndexError:
            cv_isi = np.nan
        cv_isis[ind] = cv_isi
        
    return cv_isis

def calculate_neuron_mfr(col, num_mins_per_bin, total_time):
    num_bins = np.int(total_time / num_mins_per_bin)
    col_bins = np.array_split(col, num_bins)
    mfrs = pd.Series(np.zeros(num_bins))

    for ind, col_bin in enumerate(col_bins):
        spike_times = pd.to_numeric(col_bin[col_bin.notnull()].index.values)
        try:
            spike_train = SpikeTrain(times=spike_times,
                                     t_stop=spike_times[-1],
                                     units=ns)
            mfr = mean_firing_rate(spike_train)
        except IndexError:
            mfr = np.nan
        mfrs[ind] = mfr
    mfrs *= 10**10
    return mfrs


def plot_cluster(dfs, max_time, experiment, fig_folder, medians, labs=['Firing Rate', 'CV-ISI']):
    num_mins = np.int(max_time / 60)

    if experiment == 'CIT':
        condition_lab_1 = 'Citalopram'
        condition_lab_2 = 'WAY'

    elif experiment == 'DREADD':
        condition_lab_1 = 'CNO'

    for col in range(len(dfs[0].columns)):
        # New set of plots for each column (for each cluster)
        f, a = plt.subplots(figsize=(12, 12), nrows=2)

        for ind, df in enumerate(dfs):
            # Plot Firing rate and CV ISI over time
            x = np.linspace(0, num_mins, len(df))
            y = df.iloc[:, col]
            a[ind].plot(x, y, label=('Cluster {}'.format(str(col))))

            # Plot line for median Firing rate
            line_y = np.ones(10) * medians[ind].iloc[col]
            line_x = np.linspace(1, num_mins, 10)
            a[ind].plot(line_x, line_y, linestyle='--', color='grey',
                        label='Median {lab}'.format(lab=labs[ind]))

            # Set condition indicators
            condition_indecator_y = (np.ones(2) * np.max(df.iloc[:, col])) + 1
            condition_indecator_x = np.linspace(60, num_mins, 2)
            a[ind].plot(condition_indecator_x, condition_indecator_y, linewidth=4, label=(condition_lab_1))

            # Indicate WAY if data from CIT experiment
            if num_mins > 135 and experiment == 'CIT':
                condition_indecator_x = np.linspace(120, num_mins, 2)
                condition_indecator_y = (np.ones(2) * np.max(df.iloc[:, col])) + 2
                a[ind].plot(condition_indecator_x, condition_indecator_y, linewidth=4, label=(condition_lab_2))

            a[ind].set_title('{lab} over time.\nCluster {clus}'. format(clus=df.columns[col], lab=labs[ind]))

            # Set plot aesthetics
            a[ind].set_ylabel(labs[ind])
            a[ind].set_xlabel('Time [minutes]')
            a[ind].fill_between(x, y)
            a[ind].legend()

        if not os.path.exists(fig_folder):
            os.mkdir(fig_folder)
        plt.savefig(''.join([os.path.join(fig_folder, str(col)), '.png']))
        plt.close()

C:\Users\Rory\Anaconda3\lib\site-packages\elephant\spade.py:82: UserWarning: fim.so not found in elephant/spade_src folder,you are using the python implementation of fast fca
  'you are using the python implementation of fast fca')


In [2]:
recording = '2018-05-01_01'
data_dir = r'C:\Users\Rory\raw_data\CIT_WAY\spikes_df'
fig_folder = r'C:\Users\Rory\raw_data\CIT_WAY\figures'
verbose=True

In [3]:
df = load_data(recording=recording,
                       data_dir=data_dir,
                       verbose=verbose)
df.head()

Loading data:	2018-05-01_01


,Unnamed: 0,spike_cluster,time,condition
0,8,99,0.015833,Baseline
1,14,57,0.023667,Baseline
2,15,119,0.023767,Baseline
3,19,97,0.026600,Baseline
4,31,97,0.048833,Baseline


In [4]:
df = manipulate_df(df)
df.tail()

,Unnamed: 0,spike_cluster,time,condition,spike
539313,940485,83,02:11:36.379633,WAY,1
539314,940531,6,02:11:37.217800,WAY,1
539315,940532,119,02:11:37.218033,WAY,1
539316,940533,51,02:11:37.218067,WAY,1
539317,940534,26,02:11:37.218067,WAY,1


In [5]:
max_times, n_conditions = get_condition_times(df, experiment='CIT')
print(max_times)
print(n_conditions)

[('CIT', 7199.9696), ('WAY', 7897.218067)]
2


In [6]:
df_base = df[df['condition']=='Baseline']
df_base.head()

,Unnamed: 0,spike_cluster,time,condition,spike
0,8,99,00:00:00.015833,Baseline,1
1,14,57,00:00:00.023667,Baseline,1
2,15,119,00:00:00.023767,Baseline,1
3,19,97,00:00:00.026600,Baseline,1
4,31,97,00:00:00.048833,Baseline,1


In [7]:
df_ts = create_time_series(df_base)
df_ts.head(30)

spike_cluster,0,4,6,9,13,16,17,22,24,26,...,74,83,97,98,99,103,112,113,116,119
time,,,,,,,,,,,,,,,,,,,,,
00:00:00.015833,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
00:00:00.023667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00:00:00.023767,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
00:00:00.026600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00:00:00.048833,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00:00:00.064000,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00:00:00.091900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
00:00:00.116000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
00:00:00.117467,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [171]:
x = df_ts.index.to_pytimedelta()
q = x[0]
q.seconds

0

In [135]:
cov_ts = df_ts.apply(func=calculate_neuron_cov,num_mins_per_bin=1, total_time=60)
mean_firing_rates_ts = df_ts.apply(func=calculate_neuron_mfr,num_mins_per_bin=2,total_time=60)

C:\Users\Rory\Anaconda3\lib\site-packages\numpy\core\_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
C:\Users\Rory\Anaconda3\lib\site-packages\numpy\core\_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
C:\Users\Rory\Anaconda3\lib\site-packages\numpy\core\_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\Rory\Anaconda3\lib\site-packages\scipy\stats\stats.py:999: RuntimeWarning: Mean of empty slice.
  return a.std(axis) / a.mean(axis)
C:\Users\Rory\Anaconda3\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [161]:
cov_ts.shape

(60, 29)

In [162]:
len(mean_firing_rates_ts.columns)

29

In [165]:
empty = np.zeros(len(mean_firing_rates_ts.columns))
for col in range(len(mean_firing_rates_ts.columns)):
    vals = mean_firing_rates_ts.iloc[:, col].dropna().values
    vals = np.median(vals)
    empty[col] = vals
ser = pd.DataFrame({'Mean empty)
#ser.index = mean_firing_rates_ts.index
ser.index = mean_firing_rates_ts.columns
ser

,0
spike_cluster,
0,1.638131
4,0.928395
6,1.096037
9,5.627412
13,1.907738
16,0.186198
17,3.946067
22,1.221722
24,0.073402


In [155]:
mean_firing_rates_ts.index[3]

3

In [117]:
cov_medians = cov_ts.apply(np.nanmedian)
mfr_medians = mean_firing_rates_ts.apply(np.nanmedian)

In [119]:
df_stats = pd.DataFrame(data={'Firing Rate': mfr_medians.values, 'CV ISI': cov_medians.values}, index=cov_medians.index)

In [124]:
if n_conditions == 1:
    max_time = max_times[0][1]
elif n_conditions == 2:
    max_time = max_times[0][1]

In [126]:
df_ts = create_time_series(df)
cv_isis_ts = df_ts.apply(func=calculate_neuron_cov,
                         num_mins_per_bin=2,
                         total_time=np.int(max_time/60))
mean_firing_rates_ts = df_ts.apply(func=calculate_neuron_mfr,
                                   num_mins_per_bin=2,
                                   total_time=np.int(max_time/60))

C:\Users\Rory\Anaconda3\lib\site-packages\numpy\core\_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
C:\Users\Rory\Anaconda3\lib\site-packages\numpy\core\_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
C:\Users\Rory\Anaconda3\lib\site-packages\numpy\core\_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\Rory\Anaconda3\lib\site-packages\scipy\stats\stats.py:999: RuntimeWarning: Mean of empty slice.
  return a.std(axis) / a.mean(axis)
C:\Users\Rory\Anaconda3\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [134]:
plot_cluster(dfs=[mean_firing_rates_ts, cv_isis_ts],
             max_time=max_time,
             experiment='CIT',
             fig_folder=r'C:\Users\Rory\Documents\Work\temp\rate_over_time',
             medians=[df_stats['Firing Rate'], df_stats['CV ISI']],
             labs=['Firing Rate [Hz]', 'CV-ISI'])